In [38]:
#基础数据

from tabulate import tabulate
from math import floor

charging_relic = 0.194 #充能绳-充能效率增加19.4%
onwak = 0.05 #翁瓦克-充能效率增加5%
relic_description = ["模式","默认","翁瓦克","充能绳","翁瓦克+充能绳"] #遗器佩戴模式
relic_modes = [0,onwak,charging_relic,onwak+charging_relic] #遗器佩戴模式对应的充能效率


In [39]:
#角色模块
#通用角色模板
class character():

    charging_requirement = 110 #能量上限
    additional_charging = 0 #攻击额外充能
    after_ult_additional_charging = 0 #攻击额外充能：通过伤害型终结技触发的部分
    self_recharge_per_action = 0 #每回合自充能
    charging_efficiency = 0 #充能效率
    A = 20 #释放普攻获得20点能量
    E = 30 #释放战技获得30点能量
    utimate_reset = 5 #释放终结技后回复5点能量
    additional_start_charging = 0 #开局额外获得能量

#佩拉
class Peila(character): 
    
    def __init__(self,trace_level=1,soul_level=0):
        self.name = f"佩拉,行迹等级{trace_level}"
        self.charging_requirement = 110 #释放终结技所需能量
        self.soul_level = soul_level
        self.trace_level = trace_level
        self.trace_charging_list = [5,5.5,6,6.5,7,7.5,8.1,8.7,9.3,10,10.5,11]
        self.additional_charging = self.trace_charging_list[self.trace_level-1] 
        self.after_ult_additional_charging = self.additional_charging    

#银狼
class Yinlang(character):
    
    def __init__(self,soul_level=0,trace_level=1):
        self.name = f"{soul_level}命银狼"
        self.charging_requirement = 110 #释放终结技所需能量
        self.soul_level = soul_level
        if self.soul_level >=1: #一魂银狼释放终结技之后，根据目标已有debuff回复能量，最多5*7=35点，此处按35点计算
            self.after_ult_additional_charging = 35
#停云
class Tingyun(character):
    
    def __init__(self,skill=0,soul_level=0,trace_level=1):
        self.name = f"停云,开局释放{skill}次秘技"
        self.skill = skill #开局放几次战技
        self.charging_requirement = 130 #释放终结技所需能量
        self.self_recharge_per_action = 5 #额外能力“亨通”，停云的回合开始时，停云回复5点能量，吃回能效率
        self.additional_start_charging = 50*self.skill #每放一次秘技，开局多50点能量
    
#艾丝妲
class Aisida(character):
    
    def __init__(self,soul_level=0,trace_level=1):
        self.name = f"{soul_level}命艾丝妲"
        self.charging_requirement = 120 #释放终结技所需能量
        self.soul_level = soul_level
        if self.soul_level >=1:  #一魂艾丝妲，战技多一段伤害，回能从30提升到35
            self.E += 5
        if self.soul_level >=4:
            self.charging_efficiency = 0.15  #四魂艾丝妲，被动大于2层时，充能效率提高15%，默认生效
            
#布洛妮娅
class Buluoniya(character):
    
    def __init__(self,soul_level=0,trace_level=1):
        self.name = "布洛妮娅"
        self.charging_requirement = 120 #释放终结技所需能量

In [40]:
#光锥模块
#通用光锥模板
class weapon:
    name = "未佩戴充能光锥"
    additional_charging = 0 #攻击额外回能
    after_ult_additional_charging = 0 #攻击额外回能：通过伤害型大招触发
    charging_efficiency = 0 #充能效率
    additional_start_charging = 0 #开局额外获得能量

#新手任务开始前：攻击减防敌人获得8点能量，吃回能效果。
class New_player_mission(weapon): 
    name = "新手任务开始前"
    additional_charging = 8
    after_ult_additional_charging = 8

#但战斗还未结束：充能效率提升10%
class Battle_not_end(weapon):  
    name = "但战斗还未结束"
    charging_efficiency = 0.1
    
#轮契&记忆中的模样：攻击减防敌人获得4-8点能量，吃回能效果
class Memory(weapon):
    
    def __init__(self,shadow):
        self.name = f"叠影等级{shadow}轮契/记忆中的模样"
        self.shadow = shadow
        self.additional_charging = 3+self.shadow*1
        self.additional_start_charging = self.additional_charging #开局额外获得能量
    

In [41]:
#计算充能函数
def calculate_charging_level(action_charge, action_point, total_additional_charging, character, weapon, relic_mode):

    relic_mode += (weapon.charging_efficiency + character.charging_efficiency)

    base_charging_level = (action_charge +

                           total_additional_charging * action_point +

                           character.self_recharge_per_action * action_point +

                           character.after_ult_additional_charging +

                           weapon.after_ult_additional_charging)

    charging_level = floor(base_charging_level * (1 + relic_mode) + character.utimate_reset)

    return charging_level

In [75]:
#主模块
def cycle(character=character,weapon=weapon, relic_modes=relic_modes):

    table = []

    total_additional_charging = character.additional_charging + weapon.additional_charging

 

    for action, action_charge, action_point in [("AA", 2*character.A, 2), ("AE", character.A+character.E, 2),
                                                ("EE", 2*character.E, 2), ("AAA", 3*character.A, 3),
                                                ("AAE", 2*character.A+character.E, 3),("AEE", 2*character.E+character.A, 3),
                                                ("EEE", 3*character.E, 3)]:

        action_list = [action]

        for relic_mode in relic_modes:

            charging_level = calculate_charging_level(action_charge, action_point, total_additional_charging, character, weapon, relic_mode)

            if charging_level >= character.charging_requirement:

                action_list.append(f'\033[0;32m{charging_level}\033[0m')

            else:

                action_list.append(f'\033[0;31m{charging_level}\033[0m')

        table.append(action_list)

    #开局放大
    start_list = ["开局打战技后能量"]
    able_list = ["是否支持开局放大"]
    
    for relic_mode in relic_modes:
        
        relic_mode += (weapon.charging_efficiency + character.charging_efficiency)
        
        start_level = character.charging_requirement / 2 #开局充能
        
        start_charging = floor(start_level + 
                               
                         (character.E + total_additional_charging + weapon.additional_start_charging) * (1 + relic_mode) + 
                               
                         character.additional_start_charging) #开局放E后充能
        
        if start_charging >= character.charging_requirement:
            able = "\033[0;32mYes\033[0m"
            start_charging = f'\033[0;32m{start_charging}\033[0m'
        
        else:
            able = "\033[0;31mNo\033[0m"
            start_charging = f'\033[0;31m{start_charging}\033[0m'
        
        start_list.append(start_charging)
        able_list.append(able)  
    
    table.append(start_list)
    table.append(able_list)
        
    #制表
    if weapon.name == "未佩戴充能光锥":
        print(f"以下是<{character.name}><不佩戴充能光锥>的充能明细：")
    
    else:
        print(f"以下是<{character.name}>佩戴光锥<{weapon.name}>的充能明细：")
    
    print(tabulate(table, headers=relic_description,tablefmt='grid'))

In [76]:
cycle(Peila(7),New_player_mission)

以下是<佩拉,行迹等级7>佩戴光锥<新手任务开始前>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 93     | 97       | 110      | 114             |
+------------------+--------+----------+----------+-----------------+
| AE               | 103    | 108      | 122      | 127             |
+------------------+--------+----------+----------+-----------------+
| EE               | 113    | 118      | 134      | 139             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 129    | 135      | 153      | 159             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 139    | 146      | 165      | 172             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 149    | 156      | 177      | 184             |
+--

In [69]:
cycle(Aisida(4))
cycle(Aisida(1),Memory(5))
cycle(Aisida(0),Memory(5))

以下是<4命艾丝妲><不佩戴充能光锥>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 51     | 53       | 58       | 60              |
+------------------+--------+----------+----------+-----------------+
| AE               | 68     | 71       | 78       | 81              |
+------------------+--------+----------+----------+-----------------+
| EE               | 85     | 89       | 99       | 102             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 74     | 77       | 85       | 88              |
+------------------+--------+----------+----------+-----------------+
| AAE              | 91     | 95       | 105      | 109             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 108    | 113      | 125      | 130             |
+---------

In [70]:
cycle(Yinlang(),New_player_mission) #银狼携带光锥：新手任务开始前
cycle(Yinlang(1))

以下是<0命银狼>佩戴光锥<新手任务开始前>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 69     | 72       | 81       | 84              |
+------------------+--------+----------+----------+-----------------+
| AE               | 79     | 82       | 93       | 97              |
+------------------+--------+----------+----------+-----------------+
| EE               | 89     | 93       | 105      | 109             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 97     | 101      | 114      | 119             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 107    | 112      | 126      | 131             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 117    | 122      | 138      | 144             |
+------

In [71]:
cycle(Peila(11)) #佩拉
cycle(Peila(7),New_player_mission) #佩拉，携带光锥：新手任务开始前

以下是<佩拉,行迹等级11><不佩戴充能光锥>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 76     | 80       | 90       | 93              |
+------------------+--------+----------+----------+-----------------+
| AE               | 86     | 90       | 102      | 106             |
+------------------+--------+----------+----------+-----------------+
| EE               | 96     | 101      | 114      | 118             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 107    | 112      | 126      | 131             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 117    | 122      | 138      | 144             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 127    | 133      | 150      | 156             |
+-----

In [72]:
cycle(Tingyun(1),Battle_not_end) #停云
cycle(Tingyun(1))

以下是<停云,开局释放1次秘技>佩戴光锥<但战斗还未结束>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 60     | 62       | 69       | 72              |
+------------------+--------+----------+----------+-----------------+
| AE               | 71     | 74       | 82       | 85              |
+------------------+--------+----------+----------+-----------------+
| EE               | 82     | 85       | 95       | 99              |
+------------------+--------+----------+----------+-----------------+
| AAA              | 87     | 91       | 102      | 105             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 98     | 102      | 114      | 119             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 109    | 114      | 127      | 132             |
